<a href="https://colab.research.google.com/github/Ubaid-Manzoor/Automatic-Serveillance-using-ML/blob/master/FeatureExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import copy
import time
import math
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import datasets, models, transforms 

use_gpu = torch.cuda.is_available()
if use_gpu:
  print("Using gpu")

In [ ]:
def load_model(model_type="vgg16"):
  if model_type == "vgg16":
    model = models.vgg16(pretrained=True,progress=True)
    return model.features

def preprocess_frame(frame):
  frame = np.reshape(frame,(frame.shape[2],frame.shape[0],frame.shape[1]))
  preprocess = transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
  ])

  return preprocess(transforms.functional.to_pil_image(frame)).unsqueeze(0)

  

def get_features(frame):
  vgg_model = load_model()
  processed_frame = preprocess_frame(frame)
  vgg_model.eval()
  features = vgg_model(processed_frame)
  return torch.flatten(features)


def save_features(features, path_to_save, frame_no):
  torch.save(features,os.path.join(path_to_save,str(frame_no)+".pt"))

def get_frame_to_take(total_frames,frames_to_take=500):
  frames_taken = np.array([])

  if total_frames < frames_to_take:
    return np.array(list(range(total_frames)))

  for i in range(0,total_frames):
    if(i%int(total_frames/frames_to_take) == 0):
      frames_taken = np.append(frames_taken,i)

  
  frames_to_remove = np.array([])
  frames_to_remove = frames_to_remove.astype('int32')
  
  for i in range(len(frames_taken)):
    if len(frames_to_remove) == (len(frames_taken) - frames_to_take):
      break
    if i%2 == 0:
      frames_to_remove = np.append(frames_to_remove,i)
    if len(frames_to_remove) == (len(frames_taken) - frames_to_take):
      break
    if (len(frames_taken) - i)%2 == 0:
      frames_to_remove = np.append(frames_to_remove,len(frames_taken) - i - 1)
  
  return set(np.delete(frames_taken,frames_to_remove))

def yeildframe(videopath):
  video = cv2.VideoCapture(videopath)

  frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
  frames_to_take = get_frame_to_take(frame_count)
  
  count = 0;success = True;first=True
  while success:
    success,frame = video.read()
    if success and count in frames_to_take:
      yield frame
    count += 1


def filterfolders(paths,folder_to_exclude=[]):
  return [path for path in paths \
                    if os.path.isdir(path) \
                        and \
                      path.split('/')[-1] not in folder_to_exclude ]

def filterfiles(paths):
  return [path for path in paths \
                    if os.path.isfile(path)]

def getfullpath(base_path,folders):
  return [os.path.join(base_path, folder) for folder in folders]

def convert_videoFrame_to_feature(path):
  datafolder = filterfolders(getfullpath(path,os.listdir(path)),["train",".ipynb_checkpoints"])
  os.makedirs(os.path.join(path,"features"),exist_ok=True)
  
  for classfolder in datafolder:
    class_name = classfolder.split('/')[-1]
    os.makedirs(os.path.join(path,"features",class_name),exist_ok=True)
    videopaths = getfullpath(classfolder,os.listdir(classfolder))
    
    for video in videopaths:
      video_name = video.split('/')[-1]
      os.makedirs(os.path.join(path,"features", class_name,video_name),exist_ok=True)

      for frame_no,frame in  enumerate(yeildframe(video)):
        frame_features = get_features(frame)
        save_features(frame_features,os.path.join(path,"features", class_name,video_name),frame_no)
      print(video.split('/')[-1], " Compeleted!!!")

In [ ]:
path = '/content/drive/My Drive/AutomaticServaliance Project/Data/Anomaly-Videos-Part-1'
convert_videoFrame_to_feature(path)